In [ ]:
!sudo apt update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
#Check this site for the latest download link https://www.apache.org/dyn/closer.lua/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz
!wget -q https://dlcdn.apache.org/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz
!tar xf spark-3.2.1-bin-hadoop3.2.tgz
!pip install -q findspark
!pip install pyspark
!pip install py4j

import os
import sys
# os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
# os.environ["SPARK_HOME"] = "/content/spark-3.2.1-bin-hadoop3.2"


import findspark
findspark.init()
findspark.find()

import pyspark

from pyspark.sql import DataFrame, SparkSession
from typing import List
import pyspark.sql.types as T
import pyspark.sql.functions as F

spark= SparkSession \
       .builder \
       .appName("Our First Spark Example") \
       .getOrCreate()

spark

Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:3 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:11 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,678 kB]
Get:12 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [8,765 kB]
Fetched 11.7 MB in 9s (1,360 kB/s)
Reading package lists... Done
Building dependency tree... Done
Rea

In [ ]:
from pyspark.sql import SparkSession
from google.colab import drive

# Mount Google Drive
#drive.mount('/content/drive')

# Initialize Spark Session
spark = SparkSession.builder.appName("EmployeeAttritionSentiment").getOrCreate()

# Define file paths (Change these paths as per your Google Drive structure)
new_review_df = "/content/drive/My Drive/new.csv"

# Load job review dataset
new_review_df = spark.read.csv(new_review_df, header=True, inferSchema=True)

# Display loaded data
new_review_df.show(5, truncate = False)

+------+--------------------------------------------------------+------------------------------------+------------------------------------------------------------------------------------------------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+---------+------------+----------------+--------------------+-------------------------+-----------------+-----------------+----------------+---------------------+------------------------------------------------------------------+------------+------------------+
|rating|title                                                   |

In [ ]:

spark = SparkSession.builder.appName("EmployeeAttritionSentiment").getOrCreate()
df = "/content/drive/My Drive/test.csv"
# Load CSV file into Spark DataFrame
df = spark.read.csv(df, header=True, inferSchema=True)

# Show Data Overview
df.show(5)


+-----------+---+------+----------------+----------+--------------+-----------------+----------------+------------------+--------------------+--------+------------------+----------------+--------------+--------------------+---------+------------+--------------+-----------+------------------------+------------------------+------------------+--------------------+---------+
|Employee ID|Age|Gender|Years at Company|  Job Role|Monthly Income|Work-Life Balance|Job Satisfaction|Performance Rating|Number of Promotions|Overtime|Distance from Home| Education Level|Marital Status|Number of Dependents|Job Level|Company Size|Company Tenure|Remote Work|Leadership Opportunities|Innovation Opportunities|Company Reputation|Employee Recognition|Attrition|
+-----------+---+------+----------------+----------+--------------+-----------------+----------------+------------------+--------------------+--------+------------------+----------------+--------------+--------------------+---------+------------+------

In [ ]:
import nltk
nltk.download('vader_lexicon')


[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


True

In [ ]:
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer

# Download VADER lexicon
nltk.download('vader_lexicon')

# Initialize VADER Sentiment Analyzer
sia = SentimentIntensityAnalyzer()


[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf, when
import pyspark.sql.functions as F  # Use F to avoid conflicts

from pyspark.sql.types import FloatType, StringType
from nltk.sentiment import SentimentIntensityAnalyzer

# -------------------- 1️⃣ VADER Sentiment Analysis for Text Columns --------------------
# Initialize VADER Sentiment Analyzer
sia = SentimentIntensityAnalyzer()

# UDF to Compute VADER Sentiment Score
def get_sentiment(text):
    if text is None or text.strip() == "":
        return 0.0  # Neutral Sentiment
    score = sia.polarity_scores(text)["compound"]  # Returns a score between -1 and 1
    return float(score)

# Register UDF
sentiment_udf = udf(get_sentiment, FloatType())

# Apply Sentiment Analysis to Text-Based Columns
text_columns = ["Work-Life Balance", "Job Satisfaction", "Company Reputation", "Employee Recognition"]
for col_name in text_columns:
    df = df.withColumn(f"{col_name}_Sentiment", sentiment_udf(col(col_name)))

# -------------------- 2️⃣ Convert Categorical Ratings into Numeric Sentiment Scores --------------------
# Mapping Categorical Ratings to Sentiment Scores
rating_map = {
    "Excellent": 1.0,
    "Good": 0.7,
    "Medium": 0.3,
    "Poor": -0.3,
    "Bad": -0.7
}

# UDF to Convert Categorical Ratings
def convert_rating_to_score(value):
    return rating_map.get(value, 0.0)  # Default to Neutral if value is missing

convert_rating_udf = udf(convert_rating_to_score, FloatType())

# Columns with Categorical Sentiment Ratings
categorical_columns = ["Performance Rating"]

# Convert Ratings into Numeric Scores
for col_name in categorical_columns:
    df = df.withColumn(f"{col_name}_Sentiment", convert_rating_udf(col(col_name)))

# -------------------- 3️⃣ Compute Final Weighted Sentiment Score --------------------
# Weight Assignments
weights = {
    "Work-Life Balance_Sentiment": 0.2,
    "Job Satisfaction_Sentiment": 0.2,
    "Company Reputation_Sentiment": 0.3,
    "Employee Recognition_Sentiment": 0.2,
    "Performance Rating_Sentiment": 0.1
}

# Compute Weighted Sentiment Score
weighted_expr = sum(weights[column] * F.col(column) for column in weights)
df = df.withColumn("Overall_Sentiment_Score", weighted_expr)

# -------------------- 4️⃣ Save Final DataFrame --------------------
df.write.csv("employee_sentiment_output_test.csv", header=True)

# Show Final Output
df.select("Employee ID", "Overall_Sentiment_Score").show(10)


+-----------+-----------------------+
|Employee ID|Overall_Sentiment_Score|
+-----------+-----------------------+
|      52685|   -0.02863000035285948|
|      30585|    0.22020000219345093|
|      54656|    0.14111999869346617|
|      33442|    -0.1340100049972534|
|      15667|    0.22020000219345093|
|       3496|    0.14111999869346617|
|      46775|   -0.05493000149726865|
|      72645|    0.14111999869346617|
|       4941|    0.22020000219345093|
|      65181|    0.03677999973297118|
+-----------+-----------------------+
only showing top 10 rows



In [ ]:
# Define Sentiment Categories
df = df.withColumn(
    "Overall_Sentiment_Label",
    when(col("Overall_Sentiment_Score") > 0.05, "Positive")
    .when(col("Overall_Sentiment_Score") < -0.05, "Negative")
    .otherwise("Neutral")
)

# Show Output
df.select("Employee ID", "Overall_Sentiment_Score", "Overall_Sentiment_Label").show(10)
df.show()

+-----------+-----------------------+-----------------------+
|Employee ID|Overall_Sentiment_Score|Overall_Sentiment_Label|
+-----------+-----------------------+-----------------------+
|      52685|   -0.02863000035285948|                Neutral|
|      30585|    0.22020000219345093|               Positive|
|      54656|    0.14111999869346617|               Positive|
|      33442|    -0.1340100049972534|               Negative|
|      15667|    0.22020000219345093|               Positive|
|       3496|    0.14111999869346617|               Positive|
|      46775|   -0.05493000149726865|               Negative|
|      72645|    0.14111999869346617|               Positive|
|       4941|    0.22020000219345093|               Positive|
|      65181|    0.03677999973297118|                Neutral|
+-----------+-----------------------+-----------------------+
only showing top 10 rows

+-----------+---+------+----------------+----------+--------------+-----------------+----------------+----

In [ ]:
# Define Sentiment Categories
df = df.withColumn(
    "Overall_Sentiment_Label",
    when(col("Overall_Sentiment_Score") > 0.05, "Positive")
    .when(col("Overall_Sentiment_Score") < -0.05, "Negative")
    .otherwise("Neutral")
)

# Show Output
df.select("Employee ID", "Overall_Sentiment_Score", "Overall_Sentiment_Label").show(10)


+-----------+-----------------------+-----------------------+
|Employee ID|Overall_Sentiment_Score|Overall_Sentiment_Label|
+-----------+-----------------------+-----------------------+
|      52685|   -0.02863000035285948|                Neutral|
|      30585|    0.22020000219345093|               Positive|
|      54656|    0.14111999869346617|               Positive|
|      33442|    -0.1340100049972534|               Negative|
|      15667|    0.22020000219345093|               Positive|
|       3496|    0.14111999869346617|               Positive|
|      46775|   -0.05493000149726865|               Negative|
|      72645|    0.14111999869346617|               Positive|
|       4941|    0.22020000219345093|               Positive|
|      65181|    0.03677999973297118|                Neutral|
+-----------+-----------------------+-----------------------+
only showing top 10 rows



In [ ]:
df.drop("Work-Life Balance_Sentiment","Job Satisfaction_Sentiment","Company Reputation_Sentiment","Employee Recognition_Sentiment","Performance Rating_Sentiment").show()

+-----------+---+------+----------------+----------+--------------+-----------------+----------------+------------------+--------------------+--------+------------------+-----------------+--------------+--------------------+---------+------------+--------------+-----------+------------------------+------------------------+------------------+--------------------+---------+-----------------------+-----------------------+
|Employee ID|Age|Gender|Years at Company|  Job Role|Monthly Income|Work-Life Balance|Job Satisfaction|Performance Rating|Number of Promotions|Overtime|Distance from Home|  Education Level|Marital Status|Number of Dependents|Job Level|Company Size|Company Tenure|Remote Work|Leadership Opportunities|Innovation Opportunities|Company Reputation|Employee Recognition|Attrition|Overall_Sentiment_Score|Overall_Sentiment_Label|
+-----------+---+------+----------------+----------+--------------+-----------------+----------------+------------------+--------------------+--------+---

In [ ]:
# Convert PySpark DataFrame to Pandas DataFrame
pandas_df = df.toPandas()

# Define the path where you want to save the file in your Google Drive
file_path = "/content/drive/My Drive/sentiment_data_test.csv"

# Save DataFrame as CSV
pandas_df.to_csv(file_path, index=False)

print(f"File saved to: {file_path}")


File saved to: /content/drive/My Drive/sentiment_data_test.csv


In [ ]:
new_review_df.show(5, truncate = False)

In [ ]:
df_10 = new_review_df.limit(10)
df_10.cache()

DataFrame[rating: string, title: string, status: string, pros: string, cons: string, Recommend: int, CEO Approval: int, Business Outlook: int, Career Opportunities: int, Compensation and Benefits: int, Senior Management: int, Work/Life Balance: int, Culture & Values: int, Diversity & Inclusion: int, firm_link: string, date: string, job: string]

In [ ]:
df_10.show()

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf, col
from pyspark.sql.types import StringType
from transformers import pipeline

# Initialize Spark Session
spark = SparkSession.builder.appName("SentimentAnalysis").getOrCreate()

# Load Pre-trained Sentiment Model (Efficient for Large Datasets)
sentiment_pipeline = pipeline("sentiment-analysis", model="cardiffnlp/twitter-roberta-base-sentiment-latest")

# Define a PySpark UDF for Sentiment Analysis
def get_sentiment(pros, cons):
    try:
        combined_text = f"{pros}. {cons}"
        result = sentiment_pipeline(combined_text)[0]
        return result["label"]  # Returns 'POSITIVE', 'NEGATIVE', or 'NEUTRAL'
    except Exception as e:
        return "ERROR"  # Handle errors gracefully

# Register the UDF with PySpark
sentiment_udf = udf(get_sentiment, StringType())

# Apply UDF in Small Batches to Avoid Memory Issues
sentiment_df = new_review_df.withColumn("overall_sentiment", sentiment_udf(col("pros"), col("cons")))

# Show Final DataFrame (First 10 Rows)
sentiment_df.show(10, truncate=False)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/929 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/501M [00:00<?, ?B/s]

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/501M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

Device set to use cpu


+------+--------------------------------------------------------+------------------------------------+-----------------------------------------------------------------------------------------------------------------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+---------+------------+----------------+--------------------+-------------------------+-----------------+-----------------+----------------+---------------------+------------------------------------------------------------------+------------+-------------------------------+-----------------+
|rating|title    